In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import gc
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

SAMPLE_SIZE = 0.55
%matplotlib inline

In [2]:
avisos = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisosONEHOT.csv")
postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulaciones.csv")
postulantes = pd.read_csv("Data/fiuba_entrenamiento/pocho/postulantesONEHOT.csv")

avisos['nombre_area']=avisos['nombre_area'].astype('category')
avisos_normales = pd.read_csv("Data/fiuba_entrenamiento/pocho/avisos.csv")
avisos_normales = avisos_normales.loc[:,["nombre_area","idaviso","tipo_de_trabajo","nivel_laboral"]]

vistas = pd.read_csv("Data/fiuba_entrenamiento/pocho/vistas.csv")

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [3]:

avisos_normales.head()
    

,nombre_area,idaviso,tipo_de_trabajo,nivel_laboral
0,Comercial,8725750,Full-time,Senior / Semi-Senior
1,Transporte,1000610287,Full-time,Senior / Semi-Senior
2,Producción,1000872556,Full-time,Senior / Semi-Senior
3,Ventas,1001135716,Full-time,Senior / Semi-Senior
4,Ventas,1001326344,Full-time,Senior / Semi-Senior


In [4]:
vistas['cantidad']=0
vistas2=vistas.groupby(['idaviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas2.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,1
1,48375,RwVdKR,1
2,169730,1KjXB,1
3,169730,2AKzxa,1
4,169730,6LJ64,1


In [5]:
vistas=vistas.groupby(['idpostulante'],as_index=False).agg({'cantidad':'count'});
vistas.head()

,idpostulante,cantidad
0,0002q,3
1,0005E,25
2,000R8,4
3,001XE,12
4,003k9,26


In [6]:
vistas3=pd.merge(vistas2,vistas,on='idpostulante',how='left')
vistas3.head()

,idaviso,idpostulante,cantidad_x,cantidad_y
0,18,BolNL,1,20
1,48375,RwVdKR,1,10
2,169730,1KjXB,1,34
3,169730,2AKzxa,1,1
4,169730,6LJ64,1,10


In [7]:
vistas3['cantidad']=vistas3['cantidad_x']/vistas3['cantidad_y']
vistas3.drop(['cantidad_x','cantidad_y'],axis=1,inplace=True)
vistas3.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,0.050000
1,48375,RwVdKR,0.100000
2,169730,1KjXB,0.029412
3,169730,2AKzxa,1.000000
4,169730,6LJ64,0.100000


In [8]:
vistas=vistas.rename(columns={'cantidad': 'vistas_postulante'})
vistas2=vistas2.rename(columns={'cantidad': 'vistas_al_aviso'})
vistas_final = pd.merge(vistas3,avisos_normales, on= "idaviso", how = "left")
vistas_final.head()

,idaviso,idpostulante,cantidad,nombre_area,tipo_de_trabajo,nivel_laboral
0,18,BolNL,0.050000,NaN,NaN,NaN
1,48375,RwVdKR,0.100000,NaN,NaN,NaN
2,169730,1KjXB,0.029412,NaN,NaN,NaN
3,169730,2AKzxa,1.000000,NaN,NaN,NaN
4,169730,6LJ64,0.100000,NaN,NaN,NaN


In [9]:
no_postulaciones = pd.read_csv("Data/fiuba_entrenamiento/pocho/no-postulaciones/azar_total.csv")
postulaciones=postulaciones.append(no_postulaciones.sample(len(postulaciones)*3//4), ignore_index=True)
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
postulaciones = postulaciones.sample(int(round(len(postulaciones)*SAMPLE_SIZE)))
no_postulaciones = 0
gc.collect()

C:\Users\Pocho\Anaconda3\lib\site-packages\pandas\core\frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)


46

In [10]:
vistas_final["vistas_por_area"]=0
vistas_final["vistas_por_tipo"]=0
vistas_final["vistas_por_nivel"]=0

vistas_area = vistas_final.groupby(['idpostulante','nombre_area'],as_index=False).agg({'vistas_por_area':'count'})
vistas_tipo= vistas_final.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'vistas_por_tipo':'count'})
vistas_nivel=vistas_final.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'vistas_por_nivel':'count'})

vistas_final = 0
gc.collect()

7

In [11]:
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [12]:
postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones = pd.merge(postulaciones,avisos_normales, on = "idaviso", how = "left")
postulaciones.drop("nombre_area_y", axis = 1, inplace=True)
postulaciones.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)
postulaciones=pd.merge(postulaciones,vistas,on=['idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas2,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas3,on=['idaviso','idpostulante'],how='left')
postulaciones=pd.merge(postulaciones,vistas_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
postulaciones['vistas_por_area']=postulaciones['vistas_por_area'].fillna(0)
postulaciones['vistas_por_tipo']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones['vistas_por_nivel']=postulaciones['vistas_por_tipo'].fillna(0)
postulaciones["vistas_al_aviso"] = postulaciones["vistas_al_aviso"].fillna(0)
postulaciones['lo_vio']=(postulaciones['vistas_al_aviso']>0)
postulaciones.cantidad = postulaciones.cantidad.fillna(0)
postulaciones.vistas_postulante = postulaciones.vistas_postulante.fillna(0)

postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,nivel_laboral_Senior / Semi-Senior,tipo_de_trabajo,nivel_laboral,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio
924856,1112281113,Yj5NkMV,1,51.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,19.0,0.0,0.000000,1.0,13.0,13.0,False
4329527,1112457398,kP6XPZW,0,27.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,26.0,0.0,0.000000,0.0,24.0,24.0,False
3055887,1112261429,vVe6Zxo,0,25.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,6.0,0.0,0.000000,0.0,6.0,6.0,False
5649314,1112192850,zva3rME,0,34.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,2.0,0.0,0.000000,0.0,1.0,1.0,False
1924035,1112380720,1QW3kB,1,42.0,False,True,False,False,False,False,...,False,Full-time,-,71.0,0.0,0.000000,0.0,53.0,53.0,False
310876,1112361177,qewDY2k,1,24.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,71.0,1.0,0.014085,22.0,56.0,56.0,True
2780223,1112349664,OqrJeQW,0,200.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,35.0,0.0,0.000000,1.0,26.0,26.0,False
2218021,1112412960,MVreBG9,1,29.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,113.0,1.0,0.008850,10.0,83.0,83.0,True
6037143,1112466207,ZDPmLXb,0,34.0,False,True,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,3.0,0.0,0.000000,0.0,2.0,2.0,False
2688268,1112326374,3N2PvMQ,0,23.0,True,False,False,False,False,False,...,True,Full-time,Senior / Semi-Senior,0.0,0.0,0.000000,0.0,0.0,0.0,False


In [13]:
postulaciones_area = postulaciones.groupby(['idpostulante','nombre_area'],as_index=False).agg({'sepostulo':'count'})
postulaciones_tipo= postulaciones.groupby(['idpostulante','tipo_de_trabajo'],as_index=False).agg({'sepostulo':'count'})
postulaciones_nivel =postulaciones.groupby(['idpostulante','nivel_laboral'],as_index=False).agg({'sepostulo':'count'})
postulaciones_area=postulaciones_area.rename(columns={'sepostulo':'postulaciones_misma_area'})
postulaciones_tipo=postulaciones_tipo.rename(columns={'sepostulo':'postulaciones_mismo_tipo'})
postulaciones_nivel=postulaciones_nivel.rename(columns={'sepostulo':'postulaciones_mismo_nivel'})

In [14]:
postulaciones.dropna(axis = 1, inplace=True)

In [15]:
postulaciones_area.head()

,idpostulante,nombre_area,postulaciones_misma_area
0,0z5Dmrd,Ingeniería Eléctrica y Electrónica,1
1,0z5Dmrd,Programación,1
2,0z5Dmrd,Selección,1
3,0z5Dmrd,Tecnologia / Sistemas,1
4,0z5Dmrd,Ventas,1


In [16]:
postulaciones=pd.merge(postulaciones,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
postulaciones=pd.merge(postulaciones,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')

postulaciones['postulaciones_misma_area']=postulaciones['postulaciones_misma_area'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_nivel']=postulaciones['postulaciones_mismo_nivel'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones['postulaciones_mismo_tipo']=postulaciones['postulaciones_mismo_tipo'].fillna(0).apply(lambda x: x if(x<=0) else x-1)
postulaciones.drop(["tipo_de_trabajo","nivel_laboral"], axis = 1, inplace=True)



In [17]:
#mean encoding
area_mean = postulaciones.groupby("nombre_area").agg({"sepostulo":"mean"})
area_mean.head()

means = area_mean.to_dict()["sepostulo"]
postulaciones.nombre_area = postulaciones.nombre_area.replace(means)
postulaciones.sample(10)

,idaviso,idpostulante,sepostulo,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,vistas_por_area,vistas_por_tipo,vistas_por_nivel,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel
5067488,1111703294,X9kR568,0,26.0,False,True,False,False,False,False,...,2.0,0.0,0.000000,0.0,2.0,2.0,False,0,21,14
1281487,1112410888,ekvEW9r,1,22.0,False,True,False,False,False,False,...,6.0,1.0,0.166667,1.0,6.0,6.0,True,0,12,1
5131724,1112445236,eJ2Pj4,0,37.0,True,False,False,False,False,False,...,77.0,0.0,0.000000,1.0,63.0,63.0,False,1,46,35
1724053,1112368214,mzVNXLO,1,22.0,True,False,False,False,False,False,...,43.0,1.0,0.023256,4.0,2.0,2.0,True,1,0,16
1872585,1112347101,6zMwO3,1,33.0,True,False,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,3,28,10
2618033,1112343014,W9VM5ol,1,37.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,2,15,8
1541257,1112268124,X9PrNbK,1,29.0,True,False,False,False,False,False,...,15.0,0.0,0.000000,2.0,5.0,5.0,False,4,9,10
411737,1112327959,0zreM5r,0,39.0,False,True,False,False,False,False,...,8.0,0.0,0.000000,0.0,7.0,7.0,False,0,151,92
775733,1112281164,qek061X,1,23.0,False,True,False,False,False,False,...,0.0,0.0,0.000000,0.0,0.0,0.0,False,0,14,3
1135994,1112323889,ZBNEZR,1,40.0,False,True,False,True,False,False,...,16.0,0.0,0.000000,2.0,15.0,15.0,False,5,28,5


In [18]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')


x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']

postulaciones=0

gc.collect()

Train:  5513158 Test:  612574


91

In [19]:
model = XGBClassifier(objective='binary:logistic',max_depth=18,learning_rate=0.1,subsample=0.8,colsample_bytree=0.8,n_estimators=1200,silent=False,n_jobs=-1,verbose=1)
dt=model.fit(x_train,y_train,eval_set=[(x_test,y_test)],verbose=True,early_stopping_rounds=5)

[0]	validation_0-error:0.111843
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.107109
[2]	validation_0-error:0.105579
[3]	validation_0-error:0.10435
[4]	validation_0-error:0.101514
[5]	validation_0-error:0.100909
[6]	validation_0-error:0.100008
[7]	validation_0-error:0.099697
[8]	validation_0-error:0.099475
[9]	validation_0-error:0.099075
[10]	validation_0-error:0.098778
[11]	validation_0-error:0.098316
[12]	validation_0-error:0.097867
[13]	validation_0-error:0.097485
[14]	validation_0-error:0.09702
[15]	validation_0-error:0.096547
[16]	validation_0-error:0.095967
[17]	validation_0-error:0.095533
[18]	validation_0-error:0.095301
[19]	validation_0-error:0.094697
[20]	validation_0-error:0.09431
[21]	validation_0-error:0.094248
[22]	validation_0-error:0.094002
[23]	validation_0-error:0.09392
[24]	validation_0-error:0.093613
[25]	validation_0-error:0.093176
[26]	validation_0-error:0.092977
[27]	validation_0-error:0.092786
[28]	validation_0-error:0

In [20]:
from sklearn.metrics import roc_auc_score
x_final=x_test[features]
y_final=model.predict_proba(x_final)
y_pred=[]
for elem in y_final:
    y_pred.append(elem[1])
score=roc_auc_score(y_test, y_pred)
print(score)

0.9788613147849806


In [21]:
print("dale mono")

dale mono


In [22]:
joblib.dump(model, "Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean3.pkl")

['Data/fiuba_entrenamiento/pocho/modelos/XGBoostMean3.pkl']

In [23]:
prediccion = pd.read_csv("Data/fiuba_entrenamiento/test_final_100k.csv")
prediccion = pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
prediccion = pd.merge(prediccion,avisos,on='idaviso',how='inner')
prediccion = pd.merge(prediccion,avisos_normales,on='idaviso',how='inner')
prediccion.drop("nombre_area_y", axis = 1, inplace=True)
prediccion.rename({"nombre_area_x":"nombre_area"},axis=1,inplace=True)

prediccion = pd.merge(prediccion,vistas,on='idpostulante',how='left')
prediccion.sample(10)


prediccion=pd.merge(prediccion,vistas2,on=['idaviso','idpostulante'],how='left')
prediccion=pd.merge(prediccion,vistas3,on=['idaviso','idpostulante'],how='left')
prediccion['cantidad']=prediccion['cantidad'].fillna(0)
prediccion['vistas_postulante']=prediccion['vistas_postulante'].fillna(0)
prediccion['vistas_al_aviso']=prediccion['vistas_al_aviso'].fillna(0)
prediccion['lo_vio']=(prediccion['vistas_al_aviso']>0)

prediccion=pd.merge(prediccion,postulaciones_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,postulaciones_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,postulaciones_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion=pd.merge(prediccion,vistas_area,on=['idpostulante','nombre_area'],how='left')
prediccion=pd.merge(prediccion,vistas_tipo,on=['idpostulante','tipo_de_trabajo'],how='left')
prediccion=pd.merge(prediccion,vistas_nivel,on=['idpostulante','nivel_laboral'],how='left')
prediccion['vistas_por_area']=prediccion['vistas_por_area'].fillna(0)
prediccion['vistas_por_tipo']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['vistas_por_nivel']=prediccion['vistas_por_tipo'].fillna(0)
prediccion['postulaciones_misma_area']=prediccion['postulaciones_misma_area'].fillna(0)
prediccion['postulaciones_mismo_nivel']=prediccion['postulaciones_mismo_nivel'].fillna(0)
prediccion['postulaciones_mismo_tipo']=prediccion['postulaciones_mismo_tipo'].fillna(0)
prediccion.nombre_area = prediccion.nombre_area.replace(means)
prediccion.sample(10)

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
18364,75034,1112463933,JBmlMbO,23.0,True,False,False,False,False,False,...,131.0,1.0,0.007634,True,5.0,16.0,4.0,18.0,76.0,76.0
96818,96491,1112494241,6VoL3j,35.0,False,True,False,False,False,False,...,5.0,0.0,0.000000,False,1.0,7.0,4.0,0.0,5.0,5.0
1574,86576,1112466769,dY5lEBP,25.0,True,False,False,False,False,False,...,0.0,0.0,0.000000,False,0.0,5.0,1.0,0.0,0.0,0.0
46887,2377,1111243499,5pMo4G,29.0,True,False,False,False,False,False,...,152.0,0.0,0.000000,False,1.0,51.0,1.0,1.0,116.0,116.0
46371,46351,1112429944,owaZbPL,26.0,True,False,False,False,False,False,...,7.0,0.0,0.000000,False,0.0,20.0,13.0,0.0,5.0,5.0
84435,33197,1112400909,5NJY3N,46.0,False,True,False,False,False,False,...,3.0,0.0,0.000000,False,0.0,3.0,3.0,0.0,1.0,1.0
60485,8661,1111788143,1jdLd3,29.0,False,True,False,False,False,False,...,44.0,0.0,0.000000,False,0.0,10.0,0.0,0.0,41.0,41.0
34801,1227,1110419747,EzEvGNN,25.0,False,True,False,False,False,False,...,733.0,1.0,0.001364,True,14.0,397.0,277.0,26.0,641.0,641.0
50309,34850,1112409388,El5z1z,31.0,True,False,False,False,False,False,...,14.0,1.0,0.071429,True,0.0,9.0,8.0,2.0,10.0,10.0
73850,19925,1112243416,edJ9pE,58.0,True,False,False,False,False,False,...,1.0,0.0,0.000000,False,0.0,5.0,1.0,0.0,1.0,1.0


In [24]:
prediccion.head()

,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_postulante,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,26.0,0.0,0.0,False,0.0,1.0,1.0,3.0,20.0,20.0
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,4.0,0.0,0.0,False,3.0,26.0,0.0,0.0,3.0,3.0
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,10.0,0.0,0.0,False,1.0,6.0,1.0,1.0,6.0,6.0
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,0.0,False,0.0,4.0,2.0,0.0,0.0,0.0
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,1.0,0.0,0.0,False,0.0,6.0,0.0,0.0,1.0,1.0


In [25]:
x_final=prediccion[features]
y_final=model.predict_proba(x_final)
y_final2=[]
print(model.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

prediccion['sepostulo']=y_final2
prediccion.head()


[0 1]
100000
100000


,id,idaviso,idpostulante,edad,sexo_FEM,sexo_MASC,sexo_NO_DECLARA,Doctorado_Abandonado,Doctorado_En Curso,Doctorado_Graduado,...,vistas_al_aviso,cantidad,lo_vio,postulaciones_misma_area,postulaciones_mismo_tipo,postulaciones_mismo_nivel,vistas_por_area,vistas_por_tipo,vistas_por_nivel,sepostulo
0,0,739260,6M9ZQR,42.0,True,False,False,False,False,True,...,0.0,0.0,False,0.0,1.0,1.0,3.0,20.0,20.0,0.031262
1,1,739260,6v1xdL,30.0,False,True,False,False,False,False,...,0.0,0.0,False,3.0,26.0,0.0,0.0,3.0,3.0,0.248902
2,2,739260,ezRKm9,36.0,True,False,False,False,False,False,...,0.0,0.0,False,1.0,6.0,1.0,1.0,6.0,6.0,0.034602
3,3,758580,1Q35ej,68.0,False,True,False,False,False,False,...,0.0,0.0,False,0.0,4.0,2.0,0.0,0.0,0.0,0.002818
4,4,758580,EAN4J6,32.0,True,False,False,False,False,False,...,0.0,0.0,False,0.0,6.0,0.0,0.0,1.0,1.0,0.001205


In [26]:
prediccion=prediccion[['id','sepostulo']]
print(prediccion.sepostulo.mean())
prediccion.head()

0.5031263074647443


,id,sepostulo
0,0,0.031262
1,1,0.248902
2,2,0.034602
3,3,0.002818
4,4,0.001205


In [27]:
prediccion.to_csv("Data/fiuba_entrenamiento/pocho/predicciones/xgbmean3.csv",index=False)